In [ ]:
!pip install hyperas

In [2]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from random import randint
from keras import backend as K
import os
import numpy as np
from sklearn.utils import shuffle
from hyperopt import Trials, STATUS_OK, tpe
import tensorflow as tf
K.set_image_data_format('channels_first')

In [3]:
os.chdir('/content/drive/My Drive/Colab Notebooks/synopsys-2022/models/regression')

In [4]:
def create_model(X_train, Y_train, X_test, Y_test):
    W_maxnorm = 3
    DROPOUT = {{choice([0.3,0.5,0.7])}}
    # DROPOUT = 0.3

    model = Sequential()
    model.add(Conv2D(32, (1,3), padding='same', input_shape=(20, 1, 20),activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(MaxPool2D(pool_size=(1, 2),strides=(1,2)))
    model.add(Flatten())

    model.add(Dense(16,activation='relu'))
    model.add(Dropout(DROPOUT))
    #model.add(Dense(2,W_constraint=maxnorm(W_maxnorm)))
    #model.add(Activation('softmax'))
    model.add(Dense(1))

    myoptimizer = RMSprop(lr={{choice([0.01,0.001,0.0001,1e-5,1e-1])}}, rho=0.9, epsilon=1e-06)
    # myoptimizer = RMSprop(lr=0.01, rho=0.9, epsilon=1e-06)
    #mylossfunc = 'binary_crossentropy'
    mylossfunc='mean_squared_error'
    model.compile(loss=mylossfunc, optimizer=myoptimizer)
    # print(model.summary())

    result = model.fit(X_train, Y_train, batch_size=100, epochs=5,validation_split=0.1,verbose=False)

    val_loss = np.amax(result.history['val_loss'])
    # model_arch = 'MODEL_ARCH'
    # bestaccfile = join('TOPDIR',model_arch,model_arch+'_hyperbestacc')
    # reportAcc(acc,score,bestaccfile)

    return {'loss': val_loss, 'status': STATUS_OK, 'model': model}

In [5]:
def train_data():
  data_train = h5py.File('./data/train.h5.batch1','r')
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train)

  data_test = h5py.File('./data/test_new.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  return X_train_shuffled, Y_train_shuffled, X_test, Y_test

In [6]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=train_data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_32x1_16_filt3',
                                          verbose=False)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)



100%|██████████| 10/10 [01:54<00:00, 11.41s/it, best loss: 0.27804774045944214]


In [8]:
X_train, Y_train, X_test, Y_test = train_data()
best_model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=0.1,verbose=True)

Epoch 1/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3885 - val_loss: 0.3369
Epoch 2/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3856 - val_loss: 0.3247
Epoch 3/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3892 - val_loss: 0.3404
Epoch 4/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3860 - val_loss: 0.3296
Epoch 5/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3864 - val_loss: 0.3243
Epoch 6/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3858 - val_loss: 0.3333
Epoch 7/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3861 - val_loss: 0.3349
Epoch 8/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3854 - val_loss: 0.3310
Epoch 9/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3864 - val_loss: 0.3234
Epoch 10/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3817 - val_loss: 0.3299

In [9]:
best_model.evaluate(X_test, Y_test)

710/710 [==============================] - 1s 2ms/step - loss: 0.4124


0.41239747405052185

In [10]:
best_model.save('./seq_32x1_16_filt3')

INFO:tensorflow:Assets written to: ./seq_32x1_16_filt3/assets
